# Primera interaccion con un LLM desde VSCode



In [5]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from langchain_ollama.llms import OllamaLLM

# Llamada de prueba a nuestro modelo local

In [ ]:

model = OllamaLLM(model="llama3.2")

model.invoke("What is LangChain?")

## Comenzando nuestro primer proyecto

In [7]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers) # obtener el sitio web
        soup = BeautifulSoup(response.content, 'html.parser') # formatear como HTML
        self.title = soup.title.string if soup.title else "No title found" # configurar 'title' en caso de que haya
        for irrelevant in soup.find_all(['script', 'style', 'img', 'input']): #eliminar etiquetas irrelevantes para el proyecto
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

# Probando el codigo

In [ ]:

tacodevtripa = Website("https://github.com/tacodevtripa")
print(tacodevtripa.title)
print(tacodevtripa.text)

## Tipos de Prompts


**system prompt** descripcion de la tarea especifica, asi como la manera en que esta debe ser realizada

**user prompt** -- Input por parte del usuario

In [9]:
"""En esta definicion cada detalle importa, el simple hecho de agregar una instruccion puedes cambiar por completo
el formato en el que el LLM genera su respuesta, incluyendo traduccion a otro idioma o formato especifico (JSON, YML, etc)"""

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

### Funcion para generar el formato deseado del input de usuario, incluyendo las variables para el contenido dinamico

In [10]:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(tacodevtripa))

# Mensajes

OpenAI establecio un formato para la manera en que la informacion es procesada por los LLMs, el siguiente ejemplo
es la forma mas basica, donde la estructura es una lista `[]` de diccionarios `{key:value}`, o elementos tipo llave:valor

Asi es posible para el LLMs diferencias que tipo de prompt esta procesando

In [12]:
messages = [
    {"role": "system", "content": "You are a Spanish assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [ ]:
# Llamada a olama, el resultado es devuelto una vez terminado de procesar por completo
model.invoke(messages)

In [ ]:
# El resultado es devuelto 'a trozos', por lo que se puede iterar para mostrarlo en pantalla conforme es recibido
for chunk in model.stream(messages):
    print(chunk, end=" ") #usamos el parametro de `end=" "` para sobreescribir que por defecto `print()` agregue un salto de linea

## Funcion para crear los mensajes con el formato especificado anteriormente

In [15]:
def messages_for_LLM(system_prompt, website):
    return [
        {"role": "system", "content": system_prompt}, #configuracion del system prompt
        {"role": "user", "content": user_prompt_for(website)} #configuracion del input de usuario con los datos de la pagina web
    ]

In [ ]:
messages_for_LLM(system_prompt, tacodevtripa)

# Funcion para unir todo

## Al proveer la URL del sitio web deseado:
- Se creara la instancia de dicho sitio web
- Por medio de el constructor de dicho metodo se almacena la informacion deseada
- Y se devuelve la respuesta del LLM, usando los metodos anteriores para obtener el resultado deseado

In [17]:
def summarize(url):
    website = Website(url)
    return model.invoke(messages_for_LLM(system_prompt, website))

In [ ]:
# Probando funcion
summarize("https://github.com/tacodevtripa")

In [19]:
# Metodo para modificar el formato de la respuesta con la libreria de `Markdown`
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
#Probando funcion
display_summary("https://github.com/tacodevtripa")

# Probemos mas sitios web

Este proyecto solo funcionara con sitios web muy basicos, puesto que algunos sitios web que usen frameworks como React, al renderizar los elementos conforme interactuas con la pagina, la informacion puede ser no muy precisa

E incluso algunos sitios web cuentan con alguna proteccion basica gracias a sus proveedores de DNS, como CloudFrount, y su acceso por medio de estas librerias puede no funcionar

In [ ]:
display_summary("https://www.thetimes.com/")

In [ ]:
display_summary("https://python.langchain.com/docs/introduction/")

## Ejemplo Adicional

- El `System Prompt` fue definido para crear un ayudante en la creacion de los asuntos para los correos electronicos de acuerdo a su contenido
- Por `User Prompt` pasaremos el contenido completo de un correo electronico inventado, la forma en que esta informacion llegue ya la dejo a su imaginacion
- Notese como se especifica en el System Prompt que solo provea el 'asunto' para dicho email, para poder utilizar dicho valor y guardarlo en una variable, esto puede o no funcionar dependiendo de la 'fuerza' de cada LLM, pero por lo general siempre funciona, cosa que incluso puede ser cambiada para pedir varias opciones en una sola respuesta

In [ ]:
system_prompt = """Analyze the email content and suggest a clear, concise, and effective subject line. Your task is to:

1. Read and understand the entire email content.
2. Identify the main topic or purpose of the email.
3. Determine the tone and language used in the email (e.g., formal, informal, promotional).
4. Consider the recipient's context and potential emotional resonance.

Create a subject line that accurately reflects the content of the email and is likely to capture the attention of
the recipient.

Provide nothing but the subject line, so the value can be stored in a variable, and provide 3 options"""

user_prompt = """
Dear Tacodevtripa,

We're excited to announce the launch of our new product, "EcoCycle" - a revolutionary recycling system designed to
help households and businesses reduce their waste output.

As one of our valued partners, we're offering you an exclusive 10% discount on your first purchase. Simply use the
code ECOCYCLE10 at checkout to redeem your discount.

Our team has worked tirelessly to develop EcoCycle, and we're confident it will make a significant impact in
reducing waste and promoting sustainability.

To learn more about EcoCycle and how it can benefit your organization, please click on the link below:


Best regards,
EcoCycle Team
"""

def messages_for_email_agent(system_prompt, content):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": content}
    ]
messages = messages_for_email_agent(system_prompt, user_prompt)


model.invoke(messages)